In [0]:
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import numpy as np

### A/B тестирование

В этом задании вы познакомитесь с A/B тестированием и примените полученные знания по статистике. 

Рассмотрим A/B тестирование на примере сайта. У сайта есть два дизайна - старый и новый, и мы хотим оценить, насколько новый дизайн лучше старого. Для этого пользователи сайта случайным образом разделяются на контрольную и тестовую группы. Контрольной группе показывается старая версия сайта, тестовой группе - измененная версия. Оценить изменение можно несколькими способами, самый простой - оценить конверсию. Конверсия - доля пользователей, совершивших заранее определенное действие(например подписка, нажатие на кнопку, заполнение формы).

### Описание данных

Для начала нужно загрузить данные из файла `a_b_testing.csv` при помощи функции `read_csv` из библиотеки `pandas`. В данном случае 1 - была совершена подписка на сайт, 0 - подписки не было. A - контрольная группа, B - тестовая группа.

Далее нужно выполнить следующие пункты, описание выходного формата содержится внутри каждого задания.

### Доверительный интервал

В видео про доверительный интервал мы рассмотрели, как можно аппроксимировать биномиальное распределение нормальным. В некоторых случаях параметры нормального распределения можно вывести математически и ниже мы рассмотрим как.

Представим количество пользователей как случайную величину из биномиального распределения с параметрами  `n`  - количество пользователей и `p` - вероятность конверсии или как сумму  `n`  независимых бросаний монетки. Определим следующую случайную величину:

$$Y = X_{1} + X_{2} + \dots + X_{n} , \, $$
где случайная величина $X_{i}$ имеет распределение Бернулли. Для случайной величины $Y$ математическое ожидание и дисперсия равны:

$$\mu = np, \, \sigma^{2} = np\cdot(1 - p)$$

Далее применяя центральную предельную теорему(случайные величины $X_{i}$ распределены независимо и размер выборки большой), получаем что 

$$Y \sim \mathcal{N}(np \, np\cdot(1 - p))\$$

Мы перешли от биномиального распределения к нормальному. Следующий шаг - стандартизация нормального распределения:

$$Z = \frac{Y - np}{\sqrt{np\cdot(1-p)}} \sim \mathcal{N}(0, \, 1) $$ 

Преобразуем выражение выше:

$$Z = \frac{Y - np}{\sqrt{np\cdot(1-p)}} = \frac{\frac{Y}{n} - p}{\sqrt{\frac{p(1-p)}{n}}} \sim \mathcal{N}(0, \, 1) $$

Так как среднее значение по выборке - это наблюдаемый процент конверсии, то доверительный интервал будет выглядеть следующим образом:

$${P}\left(p - z_{1-\frac{\alpha}{2}} \sqrt{\frac{p(1-p)}{n}} \le \mu \le p + z_{1-\frac{\alpha}{2}}\sqrt{\frac{p(1-p)}{n}}\right) = 1-\alpha$$

### ЗАДАНИЕ

Найдите доверительный интервал для среднего количества пользователей, совершивших подписку из контрольной выборки с уровнем значимости 95%.

#### РЕШЕНИЕ

In [0]:
data = pd.read_csv('a_b_testing.csv')
#для контрольной выборки
p = 0.41
n = 2037

z_value = stats.norm.ppf(q = 0.975)
var = np.sqrt((p*(1-p))/n)
interval = z_value * var
conf_int = (p - interval, p + interval)  
answer1 = ','.join([str(round(c,2)) for c in conf_int])

Запишите значения левой и правой границ через запятую, (c точностью до двух знаков после запятой), сохраняя приведенный порядок, в переменную `answer1`, которая будет являтся строкой.

### Задача A/B тестирования

Рассмотрим независимые выборки $X$ и $Y$ для которых есть $\mu_x$ и $\mu_y$, определяющие среднее значение распределения.

Рассматривается следующая гипотеза:
$$
H_0: \mu_x = \mu_y
$$
против альтернативы:

$$
H_1: \mu_x \ne \mu_y.
$$

Если гипотеза $H_0$ отвергается, то показатель действительно поменялся.

Также можно тест можно записать и другим способом:
$$
H_0: \mu_x \le \mu_y
$$

против альтернативы:

$$
H_1: \mu_x > \mu_y
$$

### Статистика Стьюдента ЗАДАНИЕ

Найдите значение статистики Стьюдента в предположении независимости выборок

$$
T(X, Y) = \frac{\bar{X} - \bar{Y}}{\sqrt{\frac{s_x^2}{n} + \frac{s_y^2}{m}}}
$$

`n` - размер контрольной выборки, `m`  - размер тестовой выборки

Ответ запишите в переменную `answer2` с точностью до 2 знака после запятой

### РЕШЕНИЕ

In [0]:
n = 2037
m = 1963
mu_x = 0.411880
mu_y = 0.437086

def t_stat_binomial(n, m, mu_x, mu_y):
    return (mu_x - mu_y) / np.sqrt(mu_x * (1 - mu_x) / n + mu_y * (1 - mu_y) / m)

answer2 = round(t_stat_binomial(n, m, mu_x, mu_y), 3)

### Статистика Стьюдента из библиотеки Scipy

Найдите p-value для статистики Стьюдента, используя функцию `stats.ttest_ind`.

### РЕШЕНИЕ

In [0]:
from scipy.stats import ttest_ind

X = data[data['group'] == 'A']['converted']
Y = data[data['group'] == 'B']['converted']

answer3 = ttest_ind(X, Y, equal_var=False).pvalue

Дополнительная проверка: значение статистики Стьюдента, посчитанная двумя способами, должны совпадать

Ответ запишите в переменную `answer3` с точностью до 2 знака после запятой

### Ответы

In [0]:
output = """Confidence interval:{0:.2f}
T score custom {1:.2f}
p value {2:.2f}"""
print(output.format(answer1, answer2, answer3))

Confidence interval:0.39,0.43
T score custom -1.61
p value 0.11
